为什么选择的激活函数普遍具有梯度消失的特点?  
开始的时候我一直好奇为什么选择的激活函数普遍具有梯度消失的特点，这样不就让部分神经元失活使最后结果出问题吗？后来看到一篇文章的描述才发现，正是因为模拟人脑的生物神经网络的方法。在2001年有研究表明生物脑的神经元工作具有稀疏性，这样可以节约尽可能多的能量，据研究，只有大约1%-4%的神经元被激活参与，绝大多数情况下，神经元是处于抑制状态的，因此ReLu函数反而是更加优秀的近似生物激活函数。  
所以第一个问题，抑制现象是必须发生的，这样能更好的拟合特征。  
那么自然也引申出了第二个问题，为什么sigmoid函数这类函数不行？  
1.中间部分梯度值过小（最大只有0.25）因此即使在中间部分也没有办法明显的激活，反而会在多层中失活，表现非常不好。
2.指数运算在计算中过于复杂，不利于运算，反而ReLu函数用最简单的梯度  
在第二条解决之后，我们来看看ReLu函数所遇到的问题，  
1.在负向部分完全失活，如果选择的超参数不好等情况，可能会出现过多神经元失活，从而整个网络死亡。  
2.ReLu函数不是zero-centered，即激活函数输出的总是非负值，而gradient也是非负值，在back propagate情况下总会得到与输入x相同的结果，同正或者同负，因此收敛会显著受到影响，一些要减小的参数和要增加的参数会受到捆绑限制。  
这两个问题的解决方法分别是  
1.如果出现神经元失活的情况，可以选择调整超参数或者换成Leaky ReLu 但是，没有证据证明任何情况下都是Leaky-ReLu好  
2.针对非zero-centered情况，可以选择用minibatch gradient decent 通过batch里面的正负调整，或者使用ELU(Exponential Linear Units)但是同样具有计算量过大的情况，同样没有证据ELU总是优于ReLU。  
所以绝大多数情况下建议使用ReLu。  

## 多层感知机从零开始实现

In [65]:
%matplotlib inline
import torch
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append("../")
import d2lzh1981.utils as d2l
print(torch.__version__)

1.1.0


### 数据下载

In [66]:
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size,root='D:\\Users\\Desktop\\haha\\data\\mnist')

### 定义模型参数

In [67]:
num_inputs = 784
num_outputs = 10
num_hiddens = 256

W1 = torch.tensor(np.random.normal(0,0.01,(num_inputs, num_hiddens)), dtype=torch.float)
b1 = torch.zeros(num_hiddens, dtype=torch.float)
W2 = torch.tensor(np.random.normal(0,0.01,(num_hiddens, num_outputs)), dtype=torch.float)
b2 = torch.zeros(num_outputs, dtype=torch.float)

params = [W1, b1, W2, b2]
for param in  params:
    param.requires_grad_(requires_grad=True)

### 定义激活函数

In [68]:
#torch.max(input, other, out=None) 
'''
返回两个元素的最大值。
input(Tensor) ---- 待比较张量
other(Tensor) ---- 比较张量
out(Tensor,可选的) ---- 结果张量
'''
def relu(X):
    return torch.max(input=X, other=torch.tensor(0.0))

### 定义网络

In [69]:
def net(X):
    X = X.view((-1,num_inputs))
    H = relu(torch.matmul(X,W1)+b1)
    return torch.matmul(H,W2)+b2

### 定义损失函数

In [70]:
loss = torch.nn.CrossEntropyLoss()

### 训练

In [74]:
num_epochs, lr = 5, 0.5
d2l.train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size, params, lr)


epoch 1, loss 0.0054, train acc 0.605, test acc 0.605
epoch 2, loss 0.0051, train acc 0.619, test acc 0.618
epoch 3, loss 0.0048, train acc 0.630, test acc 0.629
epoch 4, loss 0.0045, train acc 0.640, test acc 0.628
epoch 5, loss 0.0043, train acc 0.646, test acc 0.637


## pytorch实现

In [63]:
import torch
from torch import nn
from torch.nn import init
import numpy as np
import sys
sys.path.append("../")
import d2lzh1981.utils as d2l
print(torch.__version__)

1.1.0


In [64]:
#初始化模型和各个参数
num_inputs = 784
num_outputs = 10
num_hiddens = 256
batch_size = 256
net = nn.Sequential(
    d2l.FlattenLayer(),
    nn.Linear(num_inputs, num_hiddens),
    nn.ReLU(),
    nn.Linear(num_hiddens, num_outputs),
)
for params in net.parameters():
    init.normal_(params, mean=0, std=0.01)
    
#开始训练
loss = torch.nn.CrossEntropyLoss()
num_epochs = 5
optimizer = torch.optim.SGD(net.parameters(), lr=0.5)
d2l.train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size, None, None, optimizer)


epoch 1, loss 0.0031, train acc 0.707, test acc 0.739
epoch 2, loss 0.0019, train acc 0.820, test acc 0.784
epoch 3, loss 0.0017, train acc 0.842, test acc 0.834
epoch 4, loss 0.0015, train acc 0.857, test acc 0.821
epoch 5, loss 0.0014, train acc 0.865, test acc 0.840
